In [13]:
## Load our Modules. statsmodels is a new one we need to make sure is downloaded

from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [14]:
## Read in our Data

df = pd.read_csv('insurance.csv')

In [15]:
## Check the top five rows to see what we are working with

df.head(5)

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [16]:
## Check out the summary statistics for missing values or outliers

df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


In [17]:
## This function will take our categorical variables and turn them into ones and zeros

pd.get_dummies(df.sex)

,female,male
0,1,0
1,0,1
2,0,1
3,0,1
4,0,1
...,...,...
1333,0,1
1334,1,0
1335,1,0
1336,1,0


In [19]:
pd.get_dummies(df.region)

,northeast,northwest,southeast,southwest
0,0,0,0,1
1,0,0,1,0
2,0,0,1,0
3,0,1,0,0
4,0,1,0,0
...,...,...,...,...
1333,0,1,0,0
1334,1,0,0,0
1335,0,0,1,0
1336,0,0,0,1


In [11]:
## This is how we save our results from the dummy transformation into our dataset. We only need to do it for female
## as not being female implies male for this problem

df['female'] = pd.get_dummies(df.sex)['female']

In [ ]:
## peek to make sure it worked

df.head(5)

In [ ]:
## Do the same for smokers vs not smokers

df['isSmoker'] = pd.get_dummies(df.smoker)['yes']

In [ ]:
## Create a dummy for each region except for one. Including all regions would create multicollineraity.
##  We make one for SW, NE, and SE. So that when all of these are 0, it implies the person is from NW.

df['southwest'] = pd.get_dummies(df.region)['southwest']

In [ ]:
df['northeast'] = pd.get_dummies(df.region)['northeast']

In [ ]:
df['southeast'] = pd.get_dummies(df.region)['southeast']

In [ ]:
## Check again to make sure it looks good

df.head(5)

In [ ]:
## There are a few ways to get regression results in Python. This first way is with scikit-learn. This method is used
## mainly for machine learning purposes as we do not get a full print out of regression results and need to 
## ask for each component

## Let's start with a simple linear regression where we use as the Independent variable and Charges as the dependent variable

X = df[['bmi']]
y = df['charges']
model = LinearRegression().fit(X,y)

In [ ]:
## This is how to extract the residuals and the predicted values
y_pred = model.predict(X)
resid = y - y_pred

In [ ]:
## This is the slope of the linear model
model.coef_[0]

In [ ]:
## This is the intercept of the linear model
model.intercept_

In [ ]:
## This is how we get our R-Squared value. This tells us the overall fit of the model
model.score(X,y)

In [ ]:
## This is how we get a prediction for annual charges with a BMI of 30
model.predict([[30]])

In [ ]:
## Here is our scatterplot with the linear regression line
plt.scatter(X,y)
plt.plot(X,model.coef_[0]*X + model.intercept_,color='red')
plt.show()

In [ ]:
## Let's try a multiple regression problem adding in Age as an independent variable.
X = df[['bmi','age']]
y = df['charges']
model = LinearRegression().fit(X,y)

In [ ]:
## This gives us two coefficients, one for each of our independent variables
model.coef_

In [ ]:
## This is the intercept of the linear model
model.intercept_

In [ ]:
## The R^2 increased a bit. This model is looking better already.
model.score(X,y)

In [ ]:
## This is how we get a prediction for annual charges with a BMI of 30 and age of 50
model.predict([[30,50]])

In [ ]:
# Let's dive into some multiple regression using the second method. This method uses the statsmodels module. 

In [ ]:
## This is our pairplot to look for multicollinearity between the independent variables and
## linearity between the dependent and independent variables

sns.pairplot(df,x_vars=['charges','age','bmi','children'],y_vars=['charges','age','bmi','children'],corner=True)
plt.show()

In [ ]:
## This is how we fit the multiple regression model. It is a little different format than what we had before
## but the results it gives us will be cleaner. The first line you can add or subtract variables from your model.
## The second line adds a column of ones to the from of your variables, a nessecary step for this model
## The third row is our dependent variables
## You may get a warning here that has to do with the second line

X=df[['age','bmi','children','female','isSmoker','southeast','northeast','southwest']]
X = sm.add_constant(X)
y=df['charges']

In [ ]:
## This is how we check the VIFs for multicollinearity. The cutoff value we use is 5. If any of the variables
## have a VIF greater than 5, you should leave it out of your model
## This passes the multicollinearity assumptions
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]
print(vif_data)

In [ ]:
## This fits our model, get the predicted values, and the residuals
model = sm.OLS(y,X).fit()
y_pred = model.predict(X)
resid = y - y_pred

In [ ]:
## This gives us a nice model summary. No more need to grab each coefficient or R^2
print(model.summary())

In [ ]:
## This looks for constant variance and linearity. Both look pretty bad and fail the assumptions
plt.scatter(y_pred,resid,color='b')
plt.axhline(y=0, color='r', linestyle='-')
plt.show()

In [ ]:
## This looks for normality. Pretty positively skewed. Not Good!
plt.hist(resid)
plt.show()

In [ ]:
## Transform charges into Log(charges)
df["lcharges"] = np.log(df["charges"])

In [ ]:
sns.pairplot(df,x_vars=['lcharges','age','bmi','children'],y_vars=['lcharges','age','bmi','children'],corner=True)
plt.show()

In [ ]:
## Redefine our model
X=df[['age','bmi','children','female','isSmoker','southeast','northeast','southwest']]
X = sm.add_constant(X)
y=df['lcharges']

In [ ]:
## Refit the model
model = sm.OLS(y,X).fit()
y_pred = model.predict(X)
resid = y - y_pred

In [ ]:
## Get the summary
print(model.summary())

In [ ]:
## Some weird stuff going on here. Fails both linearity and constant variance assumptions
plt.scatter(y_pred,resid,color='b')
plt.axhline(y=0, color='r', linestyle='-')
plt.show()

In [ ]:
## Pretty skewed. Fails normality
plt.hist(resid)
plt.show()

In [ ]:
## Let's drop Age because it has those weird patterns with charges. Will still use log(charges) as dependent
X= df[['bmi','children','female','isSmoker','southeast','northeast','southwest']]
X = sm.add_constant(X)
y= df['lcharges']

In [ ]:
model = sm.OLS(y,X).fit()
y_pred = model.predict(X)
resid = y - y_pred
print(model.summary())

In [ ]:
## This looks better but we can see two distinct groups that have different variances
plt.scatter(y_pred,resid,color='b')
plt.axhline(y=0, color='r', linestyle='-')
plt.show()

In [ ]:
## This looks better for normality but not perfect
plt.hist(resid)
plt.show()